## Jupyter Notebook space for Passive Active NLP StageLens project

General module loading

In [1]:
!python --version
!python -m spacy download en_core_web_md

print("Downloaded")

Python 3.6.3
    100% |████████████████████████████████| 120.9MB 53.3MB/s ta 0:00:01   76% |████████████████████████▍       | 92.2MB 39.5MB/s eta 0:00:01    78% |█████████████████████████▏      | 94.8MB 39.3MB/s eta 0:00:01    86% |███████████████████████████▊    | 104.6MB 49.3MB/s eta 0:00:01    93% |█████████████████████████████▊  | 112.4MB 48.9MB/s eta 0:00:01    97% |███████████████████████████████▎| 117.9MB 38.4MB/s eta 0:00:01

    Linking successful
    /srv/venv/lib/python3.6/site-packages/en_core_web_md -->
    /srv/venv/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')

Downloaded


In [2]:
import spacy
import html
from spacy import displacy
import re
PATTERN = "[^a-zA-Z0-9_\s\.]+"
rgx = re.compile(PATTERN, re.IGNORECASE)

nlp = spacy.load('en_core_web_md')
print("Loaded models")

Loaded models


Preprocessing input text

In [30]:
tsv = """When people are helpless	I am inspired to reach out and be of service.
Rules	are guard rails. It is good and helpful to have them. When you head in the right direction you never get in touch with them, no matter how fast or slow you travel. Good rules are not in your way.
Technology	is a great tool, nice to have and can bring global humanity closer together in ways never before imagined. But if it were to disappear tomorrow, I'm of the generation that would be able to survive.
A good boss	A good boss is critical toward making a workplace environment healthy and successful!
My main problem is	getting over decades spent proving myself.
I feel sorry	for the pain that all humans and all beings on this earth must endure as part of the experience of being here.
If I were in charge	i would change that by facilitating change that made everyone in charge ... to the degree of their capabilities in wordly matters, but always in charge of their interior and their right to decide for themselves what is real and true for them.
At times I worry about	big, big existential questions, and little microscopic ones... sometimes within milliseconds of one another.
Raising a family	is a profound gift. I am tending the future. I am tending the life processes. It is a place for love and life and evolution to soar.
At times I worry about	how differently I see and process things from many of the people I encounter, and whether I am wisely acting and interacting.
What I like to do best is	be a conduit. There is no I.
Being with other people	can be an isolating or a hugely enriching experience, the latter when each others presence liberates emergent thoughts or self expressions bringing forth a collective that is more than the sum of its parts
A teacher has the right to	I don't mean to be oppositional. . but this sentence stem. . .like so many others. . isn't one I can really answer outside of context. Rights. This is a wobbly construct. I'm sorry. I'm out of time otherwise I would write a bit about the possibility of rights . .the idea of having a right. . the usage of the language of rights.
Being with other people	is most often a pleasure and fascinating experience of seeing a completely different version of Self.
The past	is a construct to give form and structure to the present moment.
We could make the world a better place if	the mystery was okay to be as a question, that is to say if the question was okay to be a question and the answer could be a question and the answers and questions could be held together because they need each other and we realize a little faster that they won't end and they are growing together and rest in that.
A good boss	is a solar being ... one that brings light and eternal love without demanding anything of the others ... an energetic and gravitational field of such calm, peaceful magnitude that all are warmed and captivated by their glow ... brings a new burst life to all those who are in their presence
My conscience bothers me if	I'm not placing my feet on the path with an open heart, mind and spirit, tuning in to what's moving and showing up fully at the dance.
When I am criticized	I can embody my personality and feel a pang contrast to the story I tell myself inside the story we tell ourselves of what we think in the moment to be an absolute before it falls away into a deeper, clearer calm of letting go.
The thing I like about myself is	that here I sit in the drop of this essential moment, chuckling at this question! :-D
My main problem is	integrating emptiness with everyday life, finding meaning in meaningless given the meaningless in meaning making.
My main problem is	how my narccissim gets in the way of seeing the truth of the illusion of self.
These days, work	Is a rainbow of consciousness manifesting through each sparkling emanation called a 'me'.
Being with other people	Is a mutually mutating meeting of universes
What I like to do best is	lay back and live the window that I have becomeas if a home with no walls, the infinite moves through as if there is no solid boundary.the veil is no more and the whole field that is everywhere exists where the inside and outside have no difference.
Change is	the heartbeat of the universe... aliveness calling us forth!
When I get mad	i feel adrenaline through my human body and it is fierce and i love it because it moves me and it is no different than the Spirit which moves all.
Raising a family	Captivates my whole being, just as every window on reality sees it is held in Love beyond knowing.
If I were in charge	Is an expression of the love animating all creation, the simple joy that moves all from rock to life to mind to All. It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.
Privacy	the human formconsicousness separates from the One even for a moment."""

#arr = [
#["When people are helpless", "they need support to move forward. Helpless does not need to be a permanent descriptor, most people are able to be supported to redirect into a position which plays to their strengths."], 
#["Technology", "has been one of the most significant disruptors of the very fabric of society and has been the capstone of hope for a ‘better future’ for many; and a cornerstone of deceit for a much larger engine of institutionalised social change - the ‘economy’, that has endlessly promised to make our lives simpler, easier, more comfortable, and more enjoyable. And when I take my mobile phone out of my pocket in the morning on the way to work to ring my wife, who is also on her way to work, technology is an amazing gift that allows two people who desire ongoing connection with each other the opportunity to do so."]
#]

arr = [line.split("\t") for line in tsv.split("\n")]
print(len(arr), len(arr[0]))
ips = []
for tok1, tok2 in arr:
    original_ip = "{} {}".format(tok1, tok2)
    ip = html.unescape(original_ip)
    ip = rgx.sub(' ', ip)
    ip = re.sub('\s+', ' ', ip)
    ips.append(ip)
    
print(ips)

30 2
['When people are helpless I am inspired to reach out and be of service.', 'Rules are guard rails. It is good and helpful to have them. When you head in the right direction you never get in touch with them no matter how fast or slow you travel. Good rules are not in your way.', 'Technology is a great tool nice to have and can bring global humanity closer together in ways never before imagined. But if it were to disappear tomorrow I m of the generation that would be able to survive.', 'A good boss A good boss is critical toward making a workplace environment healthy and successful ', 'My main problem is getting over decades spent proving myself.', 'I feel sorry for the pain that all humans and all beings on this earth must endure as part of the experience of being here.', 'If I were in charge i would change that by facilitating change that made everyone in charge ... to the degree of their capabilities in wordly matters but always in charge of their interior and their right to deci

In [4]:
print("<html><body>")
for ip in ips:
    doc = nlp(ip)
    a = displacy.render(doc, style='dep') # Saved in test.html
    print("<div>{}</div>".format(a))
print("</body></html>")

<html><body>
<div><svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" id="0" class="displacy" width="6350" height="487.0" style="max-width: none; height: 487.0px; color: #000000; background: #ffffff; font-family: Arial">
<text class="displacy-token" fill="currentColor" text-anchor="middle" y="397.0">
    <tspan class="displacy-word" fill="currentColor" x="50">When</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="50">ADV</tspan>
</text>

<text class="displacy-token" fill="currentColor" text-anchor="middle" y="397.0">
    <tspan class="displacy-word" fill="currentColor" x="225">people</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="225">NOUN</tspan>
</text>

<text class="displacy-token" fill="currentColor" text-anchor="middle" y="397.0">
    <tspan class="displacy-word" fill="currentColor" x="400">are</tspan>
    <tspan class="displacy-tag" dy="2em" fill="currentColor" x="400">VERB</tspan>
</text>

<text class=

Get subtrees of all verbs. For each child token, get their children recursively IF the word is not a pronoun, 

Also ignore children who are adverb modifiers (advcl)

In [27]:
ignore_word_types = ["PRON"]
ignore_dep_types = [] #["advcl"]
def get_substrings(doc, word):
    #print("#######################################")
    #print(word.text, word.pos_, word.dep_, [w.text for w in word.children])
    # Base cases: Ignorable dependencies and words and words with no children
    if word.pos_ in ignore_word_types or word.dep_ in ignore_dep_types:
        return word.text

    if len([x for x in word.children]) == 0:
        return word.text
        
    # Recursive Step
    lefts = [get_substrings(doc, l) for l in word.lefts]
    rights = [get_substrings(doc, r) for r in word.rights]
    return ' '.join(lefts + [word.text] + rights)

def split_clauses(doc):
    op = []
    for word in doc:
        if word.pos_ == "VERB":
            simple = ' '.join([l.text for l in word.lefts] + [word.text] + [r.text for r in word.rights])
            op.append(simple)
            #get_substrings(doc, word))
    return op

def split_text(doc):
    op = []
    for sent in doc.sents:
        tm = split_clauses(sent)
        op = op + tm
    return op

for ip in ips:
    doc = nlp(ip)
    tt = split_text(doc)
    print("==================================================================================================")
    print(ip)
    for x in tt:
        print()
        print(x)

When people are helpless they need support to move forward. Helpless does not need to be a permanent descriptor most people are able to be supported to redirect into a position which plays to their strengths.

When people are helpless

are they need support .

to move forward

does

Helpless does not need be .

to be descriptor

people are able

be

to be supported

to redirect into

which plays to
Technology has been one of the most significant disruptors of the very fabric of society and has been the capstone of hope for a better future for many and a cornerstone of deceit for a much larger engine of institutionalised social change the economy that has endlessly promised to make our lives simpler easier more comfortable and more enjoyable. And when I take my mobile phone out of my pocket in the morning on the way to work to ring my wife who is also on her way to work technology is an amazing gift that allows two people who desire ongoing connection with each other the opportunity to 

In [29]:
"""Privacy	the human formconsicousness separates from the One even for a moment.""".split("\t")

['Privacy',
 'the human formconsicousness separates from the One even for a moment.']